In [44]:
import pandas as pd
hn19 = pd.read_sas('./HN19_ALL.sas7bdat', format='sas7bdat')
hn20 = pd.read_sas('./HN20_ALL.sas7bdat', format='sas7bdat')
hn21 = pd.read_sas('./HN21_ALL.sas7bdat', format='sas7bdat')


In [45]:
print(len(hn19.columns.tolist()))
print(len(hn20.columns.tolist()))
print(len(hn21.columns.tolist()))

896
843
840


In [46]:
x = ['edu'] # 1.초졸이하, 2. 중졸, 3. 고졸, 4. 대졸 나머지 아웃 
y2 = ['DE1_dg'] # 당뇨 0. 없음 1. 있음   나머지 아웃 
y3 = ['DF2_dg'] # 우울증 0. 없음 1. 있음 나머지 아웃
y4 = ['DI1_dg'] # 고혈압 ``
y5 = ['DM4_dg'] #골다공증 ``
y6 = ['DM8_dg'] # 통풍  `

#y1 = ['HE_obe'] # 비만 : 4~6 부터 비만  나머지 아웃 
` 
default = ['sex', 'age'] 

columns = default + x + y2 + y3 + y4 +y5 +y6 

hn19 = hn19[columns]
hn20 = hn20[columns]
hn21 = hn21[columns]

In [47]:
def filter(df_filtered):
    conditions = (
        (df_filtered['edu'].isin([1, 2, 3, 4])) &
        (df_filtered['DE1_dg'].isin([0, 1])) &
        (df_filtered['DF2_dg'].isin([0, 1])) &
        (df_filtered['DI1_dg'].isin([0, 1])) &
        (df_filtered['DM4_dg'].isin([0, 1])) &
        (df_filtered['DM8_dg'].isin([0, 1]))
    )
    
    return df_filtered[conditions]

# Apply the conditions to filter the DataFrame
hn19_ = filter(hn19)
hn20_ = filter(hn20)
hn21_ = filter(hn21)

In [48]:
df = pd.concat([hn19_, hn20_, hn21_])
df.sample(5)

,sex,age,edu,DE1_dg,DF2_dg,DI1_dg,DM4_dg,DM8_dg
4011,1.0,59.0,3.0,0.0,0.0,0.0,0.0,0.0
461,1.0,30.0,4.0,0.0,0.0,0.0,0.0,0.0
823,1.0,70.0,2.0,0.0,0.0,0.0,0.0,0.0
5025,1.0,45.0,4.0,0.0,0.0,0.0,0.0,0.0
2530,2.0,32.0,4.0,0.0,0.0,0.0,0.0,0.0


In [49]:
# 이산화 
columns_to_convert = ['DE1_dg', 'DF2_dg', 'DI1_dg', 'DM4_dg', 'DM8_dg']
for col in columns_to_convert:
    df[col] = df[col].astype(bool)

    
# Converting 'sex' column from 1 to 'male' and 2 to 'female'
df['sex'] = df['sex'].map({1.0: 'male', 2.0: 'female'})
df.sample(5)

,sex,age,edu,DE1_dg,DF2_dg,DI1_dg,DM4_dg,DM8_dg
1795,female,31.0,4.0,False,False,False,False,False
2971,male,39.0,3.0,False,False,False,False,False
6827,female,56.0,2.0,False,False,False,False,False
5710,male,41.0,4.0,True,False,True,False,False
977,male,76.0,1.0,False,False,True,False,False


In [50]:
df['sex'] = df['sex'].map({'female': 0, 'male': 1})

# Converting boolean columns to int
bool_cols = ['DE1_dg', 'DF2_dg', 'DI1_dg', 'DM4_dg', 'DM8_dg']
df[bool_cols] = df[bool_cols].astype(int)

# Calculating correlation
corr_matrix = df.corr()
corr_matrix

,sex,age,edu,DE1_dg,DF2_dg,DI1_dg,DM4_dg,DM8_dg
sex,1.000000,-0.029557,0.120338,0.039801,-0.083242,0.033271,-0.231427,0.121459
age,-0.029557,1.000000,-0.588584,0.275977,0.034204,0.475484,0.310685,0.049578
edu,0.120338,-0.588584,1.000000,-0.217044,-0.067715,-0.364402,-0.313972,-0.012981
DE1_dg,0.039801,0.275977,-0.217044,1.000000,0.018101,0.291109,0.075009,0.022150
DF2_dg,-0.083242,0.034204,-0.067715,0.018101,1.000000,0.034297,0.086764,0.014816
DI1_dg,0.033271,0.475484,-0.364402,0.291109,0.034297,1.000000,0.160533,0.093047
DM4_dg,-0.231427,0.310685,-0.313972,0.075009,0.086764,0.160533,1.000000,-0.015279
DM8_dg,0.121459,0.049578,-0.012981,0.022150,0.014816,0.093047,-0.015279,1.000000


In [ ]:
df_combined['HE_obe'] = df_combined['HE_obe'] >= 4.0
df_combined.sample(10)

In [55]:
# 성별에 따른 분리
male_data = df[df['sex'] == 0]
print(len(male_data))
female_data = df[df['sex'] == 1]
print(len(female_data))
# 분석할 범주형 변수 목록
categorical_columns = ['edu','DE1_dg', 'DF2_dg', 'DI1_dg', 'DM4_dg', 'DM8_dg']

# 성별에 따른 빈도수와 백분율 계산 함수
def calculate_frequency_and_percentage(data, columns):
    results = {}
    for col in columns:
        counts = data[col].value_counts()
        percentages = data[col].value_counts(normalize=True) * 100
        results[col] = pd.DataFrame({'빈도수': counts, '백분율 (%)': percentages})
    return results

# 남성과 여성 데이터에 대한 빈도수와 백분율
male_stats = calculate_frequency_and_percentage(male_data, categorical_columns)
female_stats = calculate_frequency_and_percentage(female_data, categorical_columns)



9250
7398


In [56]:
male_stats

{'edu':       빈도수    백분율 (%)
 edu                 
 4.0  3282  35.481081
 3.0  2945  31.837838
 1.0  2099  22.691892
 2.0   924   9.989189,
 'DE1_dg':          빈도수   백분율 (%)
 DE1_dg                
 0       8350  90.27027
 1        900   9.72973,
 'DF2_dg':          빈도수    백분율 (%)
 DF2_dg                 
 0       8614  93.124324
 1        636   6.875676,
 'DI1_dg':          빈도수    백분율 (%)
 DI1_dg                 
 0       6972  75.372973
 1       2278  24.627027,
 'DM4_dg':          빈도수   백분율 (%)
 DM4_dg                
 0       7943  85.87027
 1       1307  14.12973,
 'DM8_dg':          빈도수    백분율 (%)
 DM8_dg                 
 0       9206  99.524324
 1         44   0.475676}

In [57]:
female_stats

{'edu':       빈도수    백분율 (%)
 edu                 
 4.0  3047  41.186807
 3.0  2724  36.820762
 1.0   930  12.570965
 2.0   697   9.421465,
 'DE1_dg':          빈도수    백분율 (%)
 DE1_dg                 
 0       6494  87.780481
 1        904  12.219519,
 'DF2_dg':          빈도수  백분율 (%)
 DF2_dg               
 0       7165  96.8505
 1        233   3.1495,
 'DI1_dg':          빈도수    백분율 (%)
 DI1_dg                 
 0       5359  72.438497
 1       2039  27.561503,
 'DM4_dg':          빈도수    백분율 (%)
 DM4_dg                 
 0       7308  98.783455
 1         90   1.216545,
 'DM8_dg':          빈도수    백분율 (%)
 DM8_dg                 
 0       7110  96.107056
 1        288   3.892944}

In [61]:
import pandas as pd
import numpy as np
from scipy.stats import chi2_contingency

diseases = ['DE1_dg', 'DF2_dg', 'DI1_dg', 'DM4_dg', 'DM8_dg']
results = {}

for disease in diseases:
    # 교육 수준과 질환 유무에 따른 분할표 생성
    contingency_table = pd.crosstab(df['edu'], df[disease])
    # 카이제곱 검정 수행
    chi2, p, dof, expected = chi2_contingency(contingency_table)
    results[disease] = {'chi2': chi2, 'p-value': p}

results

{'DE1_dg': {'chi2': 812.2915684302951, 'p-value': 9.342363708633154e-176},
 'DF2_dg': {'chi2': 84.94444478567854, 'p-value': 2.6673564479659846e-18},
 'DI1_dg': {'chi2': 2289.263726154359, 'p-value': 0.0},
 'DM4_dg': {'chi2': 1815.1207398260071, 'p-value': 0.0},
 'DM8_dg': {'chi2': 5.209469643346246, 'p-value': 0.15708582024294582}}

In [62]:
# 교육 수준에 따라 데이터프레임 분할
low_edu_df = df[df['edu'].isin([1.0, 2.0])]
high_edu_df = df[df['edu'].isin([3.0, 4.0])]

# 각 데이터프레임에서 만성 질환의 빈도 계산
low_edu_freq = low_edu_df[['DE1_dg', 'DF2_dg', 'DI1_dg', 'DM4_dg', 'DM8_dg']].sum()
high_edu_freq = high_edu_df[['DE1_dg', 'DF2_dg', 'DI1_dg', 'DM4_dg', 'DM8_dg']].sum()

# 두 그룹 간의 차이 검증을 위한 카이제곱 검정
chi2_test_results = {}
for disease in low_edu_freq.index:
    contingency_table = np.array([[low_edu_freq[disease], high_edu_freq[disease]], 
                                  [low_edu_df.shape[0] - low_edu_freq[disease], high_edu_df.shape[0] - high_edu_freq[disease]]])
    chi2, p, dof, _ = chi2_contingency(contingency_table)
    chi2_test_results[disease] = {'chi2': chi2, 'p-value': p}

chi2_test_results

{'DE1_dg': {'chi2': 722.3452083650284, 'p-value': 4.137133074378595e-159},
 'DF2_dg': {'chi2': 72.6857113157409, 'p-value': 1.5203083703496214e-17},
 'DI1_dg': {'chi2': 2078.693247695688, 'p-value': 0.0},
 'DM4_dg': {'chi2': 1606.41124026172, 'p-value': 0.0},
 'DM8_dg': {'chi2': 2.1144055343632964, 'p-value': 0.14591870518107153}}